In [1]:
import aocd
import re
from functools import cache
import itertools
%run helper.ipynb
puzzle = aocd.models.Puzzle(year=2024, day=21)
data = puzzle.input_data

In [2]:
inputs = data.split("\n")

In [3]:
dir_map = {
    (-1, 0): "^",
    ( 1, 0): "v",
    (0, -1): "<",
    (0,  1): ">"
}

def get_all_directions(grid):
    directions = {}
    for r1 in range(len(grid)):
        for r2 in range(len(grid)):
            for c1 in range(len(grid[0])):
                for c2 in range(len(grid[0])):
                    if grid[r1][c1] == " " or grid[r2][c2] == " ":
                        continue
                    r_diff = r2 - r1
                    c_diff = c2 - c1
                    if r_diff != 0:
                        r_dir_idx = r_diff // abs(r_diff)
                        r_dir = abs(r_diff) * dir_map[(r_dir_idx, 0)]
                    else:
                        r_dir = ""
                    if c_diff != 0:
                        c_dir_idx = c_diff // abs(c_diff)
                        c_dir = abs(c_diff) * dir_map[(0, c_dir_idx)]
                    else:
                        c_dir = ""
                    dirs = []
                    if grid[r1][c2] != " ":
                        dirs.append(c_dir + r_dir)
                    if grid[r2][c1] != " ":
                        dirs.append(r_dir + c_dir)
                    directions[(grid[r1][c1], grid[r2][c2])] = dirs   
    return directions

In [4]:
keypad = Grid("789\n456\n123\n 0A", str)
remote = Grid(" ^A\n<v>", str)
keypad_directions = get_all_directions(keypad)
remote_directions = get_all_directions(remote)

In [5]:
@cache
def map_directions(code, depth, start):
    temp_code = "A" + code
    pairs = zip(temp_code[:-1], temp_code[1:])
    dirs = keypad_directions if start else remote_directions
    min_len = 0
    for p in pairs:
        if depth == 0:
            min_len += min([len(x)+1 for x in dirs[p]])
        else:
            min_len += min([map_directions(d + "A", depth-1, False) for d in dirs[p]])
    # print(temp_code, depth, start, min_len)
    return min_len

def score_input(code, num_robots):
    l = map_directions(code, num_robots, True)
    return int(code[:3])* l

In [6]:
puzzle.answer_a = sum([score_input(x, 2) for x in inputs])

In [7]:
puzzle.answer_b = sum([score_input(x, 25) for x in inputs])